In [ ]:
# #####sudoku#####
import numpy as np
import matplotlib.pyplot as plt
import networkx as nx
from collections import defaultdict
import heapq
import random

class SudokuGraph:
    """Graph structure to represent the Sudoku grid."""

    def __init__(self):
        self.graph = defaultdict(list)  # Adjacency list
        self.vertices = 81  # 9x9 grid = 81 cells

    def add_edges(self):
        """Add edges to represent row, column, and 3x3 block constraints."""
        for row in range(9):
            for col in range(9):
                index = row * 9 + col  # Cell index (0-80)

                # Same row edges
                for c in range(9):
                    if c != col:
                        self.graph[index].append(row * 9 + c)

                # Same column edges
                for r in range(9):
                    if r != row:
                        self.graph[index].append(r * 9 + col)

                # Same 3x3 block edges
                start_row, start_col = (row // 3) * 3, (col // 3) * 3
                for r in range(start_row, start_row + 3):
                    for c in range(start_col, start_col + 3):
                        if r != row or c != col:
                            self.graph[index].append(r * 9 + c)

    def plot_colored_graph(self, grid, step):
        """Visualize the Sudoku graph with vertex coloring and step title."""
        G = nx.Graph()

        # Add nodes and edges to the graph
        for vertex in range(self.vertices):
            G.add_node(vertex)

        for vertex in self.graph:
            for neighbor in self.graph[vertex]:
                G.add_edge(vertex, neighbor)

        pos = {i: (i % 9, 8 - i // 9) for i in range(81)}  # Arrange nodes in a 9x9 grid

        # Define node colors based on the grid state
        colors = []
        for i in range(9):
            for j in range(9):
                if grid[i][j] != 0:
                    colors.append('lightgreen')  # Filled cells
                else:
                    colors.append('lightcoral')  # Empty cells

        nx.draw(G, pos, with_labels=True, node_size=500, node_color=colors,
                font_size=8, font_color='black', font_weight='bold')

        # Set the graph title to indicate the current step
        plt.title(f"\nSudoku Graph Representation - Step {step}" if step > 0 else "\nSudoku Graph")
        plt.show()

def is_safe(grid, row, col, num):
    """Check if a number can be placed at a given position."""
    if num in grid[row]:  # Check row
        return False
    if num in [grid[i][col] for i in range(9)]:  # Check column
        return False

    # Check 3x3 block
    start_row, start_col = 3 * (row // 3), 3 * (col // 3)
    for r in range(start_row, start_row + 3):
        for c in range(start_col, start_col + 3):
            if grid[r][c] == num:
                return False

    return True

def get_empty_cells(grid):
    """Use a heap (priority queue) to store empty cells based on MRV heuristic."""
    empty_cells = []
    for row in range(9):
        for col in range(9):
            if grid[row][col] == 0:
                candidates = sum(1 for num in range(1, 10) if is_safe(grid, row, col, num))
                heapq.heappush(empty_cells, (candidates, row, col))  # (MRV, row, col)
    return empty_cells

def solve_sudoku(grid, graph, steps=0):
    """Solve Sudoku using backtracking and MRV heuristic with intermediate graph display."""
    empty_cells = get_empty_cells(grid)

    if not empty_cells:  # If no empty cells, puzzle is solved
        return True

    _, row, col = heapq.heappop(empty_cells)  # Get cell with minimum candidates
    print(f"Heap: {empty_cells}")  # Display heap usage

    for num in range(1, 10):
        if is_safe(grid, row, col, num):
            grid[row][col] = num  # Place the number
            steps += 1

            # Display intermediate graph every 5 steps
            if steps % 5 == 0:
                print(f"Step {steps}: Placing {num} at ({row}, {col})")
                graph.plot_colored_graph(grid, step=steps)

            # Recursively solve the rest of the grid
            if solve_sudoku(grid, graph, steps):
                return True

            # Backtrack if placement doesn't work
            grid[row][col] = 0

    return False

def plot_sudoku(grid):
    """Plot the Sudoku grid."""
    fig, ax = plt.subplots(figsize=(6, 6))
    ax.set_xlim(0, 9)
    ax.set_ylim(0, 9)

    # Draw grid lines
    for i in range(10):
        lw = 2 if i % 3 == 0 else 1
        ax.plot([i, i], [0, 9], color='black', lw=lw)
        ax.plot([0, 9], [i, i], color='black', lw=lw)

    # Fill in the numbers
    for i in range(9):
        for j in range(9):
            if grid[i][j] != 0:
                ax.text(j + 0.5, 8.5 - i, str(grid[i][j]), ha='center',
                        va='center', fontsize=16)

    ax.axis('off')
    plt.show()

def get_user_input():
    """Get a Sudoku puzzle from the user."""
    grid = []
    print("Enter your Sudoku puzzle row by row. Use 0 for empty cells.")
    for i in range(9):
        row = list(map(int, input(f"Row {i+1}: ").split()))
        if len(row) != 9 or any(num < 0 or num > 9 for num in row):
            print("Invalid input. Please enter exactly 9 numbers between 0 and 9.")
            return get_user_input()
        grid.append(row)
    return grid

def generate_filled_grid():
    """Generate a fully solved Sudoku grid using backtracking."""
    grid = [[0] * 9 for _ in range(9)]

    def fill_grid():
        """Backtracking to fill grid with a valid Sudoku solution."""
        empty_cell = next(((i, j) for i in range(9) for j in range(9) if grid[i][j] == 0), None)
        if not empty_cell:
            return True  # All cells filled successfully
        row, col = empty_cell

        # Shuffle numbers to add randomness
        numbers = list(range(1, 10))
        random.shuffle(numbers)
        for num in numbers:
            if is_safe(grid, row, col, num):
                grid[row][col] = num
                if fill_grid():
                    return True
                grid[row][col] = 0  # Backtrack

        return False

    fill_grid()
    return grid

def create_puzzle(grid, difficulty=50):
    """Remove cells to create a playable Sudoku puzzle."""
    puzzle = [row[:] for row in grid]
    cells_to_remove = difficulty
    while cells_to_remove > 0:
        row, col = random.randint(0, 8), random.randint(0, 8)
        if puzzle[row][col] != 0:
            puzzle[row][col] = 0
            cells_to_remove -= 1
    return puzzle

def main():
    """Main function."""
    choice = input("Would you like to enter your own puzzle? (y/n): ").lower()
    if choice == 'y':
        grid = get_user_input()
    elif choice == 'n':
        print("\n Using default puzzle\n")
        grid = [
            [5, 3, 0, 0, 7, 0, 0, 0, 0],
            [6, 0, 0, 1, 9, 5, 0, 0, 0],
            [0, 9, 8, 0, 0, 0, 0, 6, 0],
            [8, 0, 0, 0, 6, 0, 0, 0, 3],
            [4, 0, 0, 8, 0, 3, 0, 0, 1],
            [7, 0, 0, 0, 2, 0, 0, 0, 6],
            [0, 6, 0, 0, 0, 0, 2, 8, 0],
            [0, 0, 0, 4, 1, 9, 0, 0, 5],
            [0, 0, 0, 0, 8, 0, 0, 7, 9]
        ]
    else:
        print("\nInvalid choice. Generating a random puzzle...\n")
        filled_grid = generate_filled_grid()
        grid = create_puzzle(filled_grid)

    print("Initial Sudoku Grid:")
    plot_sudoku(grid)
    print("\n")

    # Create the Sudoku graph and add edges
    sudoku_graph = SudokuGraph()
    sudoku_graph.add_edges()
    sudoku_graph.plot_colored_graph(grid, 0)

    # Solve the Sudoku puzzle
    if solve_sudoku(grid, sudoku_graph):
        sudoku_graph.plot_colored_graph(grid, step=0)  # Final step graph
        print("\nSolved Sudoku Grid:")
        plot_sudoku(grid)
    else:
        print("No solution exists.")

if __name__ == "__main__":
    main()
